# Welcome to GRC ML Hackathon Challenge A!

This challenge is a collaborative effort to perform data science using a collection of publications.  During this challenge, we will focus on pin-fin heat exchangers. The approach used is nearly identical to an anlysis which involving the NASA Taxonomy and USAF SBIR. That notebook for that analysis has been graciously shared with us by Charles Liles (charles.a.liles@nasa.gov), and can be read here: https://visualization.larc.nasa.gov/dash/notebooks/SBIR_LDA.

The data used for Challenge A is open source, and it was graciously provided by Karsten Look (karsten.look@nasa.gov).

This notebook performs the same latent Dirichlet allocation (LDA) topic modeling on the heat exchanger dataset as in the previously posted example. Some participants will will leverage inexpensive Google Cloud Storage for hosting the data, while others may use Google CoLab (via Google Drive) or storage available on a local computer such as a laptop. If using Google Cloud Storage, you can skip directly to this section by clicking this link: <a href='#cloud_storage'>Cloud Storage Read/Write Demo'</a>. If using Google CoLab, you can skip directly to section... #TODO include section on reading data from Google CoLab. And running the notebook locally, you can reference #TODO include section for reading data locally.

We will also perform t-SNE on vectorized heat exchanger data and interactive visualization using the Python bokeh library.  This link: <a href='#tsne_bokeh'>t-SNE and bokeh Visualization'</a> can be clicked to skip directly to this section of the notebook.

## Import Software Dependencies and Define Functions

We will next import all Python libraries needed for model generation below.  We will also build a function to handle entity extraction from text, stopword removal, and snowball stemming.  Stop words are common word such as "a" and "the" which are not useful for NLP machine learning; you can read more about stop word removal here: https://en.wikipedia.org/wiki/Stop_words. Stemming is a means of transforming words to their root form. For example, the terms "radiate", "radiates", and "radiation" would all transform to a root form of "radiat" after stemming.  More information about Snowball stemming is available here: https://en.wikipedia.org/wiki/Snowball_(programming_language).

In [ ]:
# Filter deprecation warning
import warnings
warnings.filterwarnings('ignore')

# Uncomment the below lines if these dependencies are not already installed
!pip install --upgrade scispacy
!pip install --upgrade spacy
# !pip uninstall -y spacy
!pip install --upgrade gensim
!pip install --upgrade pyLDAvis
!pip install --upgrade ipykernel

# Uncomment the below line if en_core_sci_sm is not loaded on your system.
!pip install --upgrade https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

#uncomment if you need to read the data files from google cloud storage
from google.cloud import storage

import time, bokeh, pyLDAvis, scispacy, spacy, string, sys, en_core_sci_sm, nltk,\
       pandas as pd,\
       numpy as np

from gensim import similarities

from spacy.lang.en import English

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors

from bokeh.models import HoverTool, CustomJS, ColumnDataSource
from bokeh.palettes import viridis, Viridis256, magma, Turbo256, linear_palette
from bokeh.transform import factor_cmap
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

import pyLDAvis.sklearn

from nltk.corpus import stopwords  
from nltk.stem.snowball import SnowballStemmer

from scipy.spatial.distance import jensenshannon

from google.cloud import storage



In [ ]:
nlp = en_core_sci_sm.load()

# Uncomment the below line if nltk's stopwords are not already on your system.
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer('english')

output_notebook()

In [ ]:
def get_entities(temp_article):
    '''
    A function for preprocessing an article's text.  Spacy's en_core_sci_sm library is used to identify scientfic domain terms. 
    Stopword removal and snowball stemming is then used to cleanup each of term.
    Arg:
        temp_article: the string input for preprocessing
    Returns:
        temp_entities: a list of preprocessed terms
    '''
    if len(temp_article) > nlp.max_length:
        nlp.max_length = len(temp_article)
    doc = nlp(temp_article)
    temp_entities = []
    for n in doc.ents:
        temp_str = ''
        for w in n:
            if w not in stop_words:
                if str(w).isalpha():
                    temp_str += stemmer.stem(str(w)) + ' '
                else:
                    temp_str += str(w) + ' '
        temp_entities.append(temp_str.strip())
    return temp_entities

We will now move forward with our preprocessing and LDA visualizations.  If you are already familiar with this work from the previous notebook, you can skip to t-SNE/visualization by following this link: <a href='#tsne_bokeh'>t-SNE and bokeh Visualization'</a>.

The team has already cleaned up the PDF articles, with the formatted version saved to `Combined_Output.txt`. Artciles in the file are newline delimited.

# Reading the data
For this challenge, there are threee different ways you can read the data.
1. From GCP Cloud storage
2. From Google Drive
3. From your local harddrive.

## From GCP Cloud storage
To read from GCP Cloud storage, we'll use the bucket ID variable and `google.cloud.storage` to connect to the bucket, and download the file a a string. The method name `download_as_string` is a little misleading. What's returned is a bytes object, so we need to decode it using `decode("utf-8")` to convert it to a Python string. All the articles are contained within a single string, so to go to a list of articles we use `split("\n")` to split the string using newline characters as the delimiter.

To preview the data, we only show the first 500 characters for the first few articles. Be careful when printing the downloaded bytes object, string, and list within JupyterLab and on Google Cloud. Trying to print everything might result in warnings, errors, or the notebook locking up.

In [ ]:
BUCKET_ID="grc-ml-hackathon"
client = storage.Client()
bucket = client.get_bucket(BUCKET_ID)
blob = bucket.get_blob("challenge-ab/processed/Combined_Output.txt")
data = blob.download_as_string().decode("utf-8")
data = data.split("\n")

# Abbreviated pata for testing purposes
data = data[:5] + data[-5:]

# print("\n\n".join((data[0][:500], data[1][:500], data[2][:500])))
print(f"DATA LEN: {len(data)}")

## Fit Heat Exchanger Articleswith a Bag of Words Vectorizer

We will set up a bag of words vectorizer below.  This is a means of representing an article or document via the number of word occurences within it.  We will then fit this vectorizer to our heat exchanger data. 

In [15]:
bow_vector = CountVectorizer(tokenizer=get_entities, min_df=2, max_df=5, ngram_range=(1,3))
data_vectorized = bow_vector.fit_transform(data)

## Define the LDA Model and Fit to the Vectorized Data

We can now define our LDA model and fit it to our vectorized heat exchanger data.  LDA requires a user to pre-select the number of topic clusters to which the document corpus will be fit.  In this case, we will select 10 clusters for topic modeling.

In [16]:
cluster_count = 10

# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=cluster_count,               # Number of topics
                                      #doc_topic_prior=1.0,
                                      max_iter=100,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [17]:
lda_output = lda_model.fit_transform(data_vectorized)

## View Fitted Topics

Now that we have fit our LDA model, let's interactively view the fitted model's in a 2D projection below.  A user can hover over topics on the left-hand side and see the top-30 most relevant terms for the topic on the right hand side.  Users can also click on words listed on the righthand side and see their usage throughout all of the fitted model clusters.

The interactive user-interface below helps a user intuitively understand goodness of fit from the original document corpus.

In [18]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, bow_vector, mds='tsne', sort_topics=False)
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1   0.009546
1       0.648955  71.707100       2        1   0.009546
2      71.019684   9.971006       3        1   0.009546
3       3.785981  33.185135       4        1  44.251179
4     -42.666500  -9.222070       5        1   0.009546
5      -3.621689 -43.526516       6        1  17.538028
6      -3.632402  -4.171708       7        1  25.164870
7     -36.584400  35.804413       8        1   6.021223
8      44.391380  50.615383       9        1   0.009546
9      40.235809 -31.591665      10        1   6.976969, topic_info=                 Term       Freq      Total Category  logprob  loglift
1514          optimum  71.000000  71.000000  Default  30.0000  30.0000
311           condens  88.000000  88.000000  Default  29.0000  29.0000
264          clearanc  42.000000  42.000000  Default  28.0000  28.0000
2244             tube  75.000000  75.000000  Default  27.0000  27.0000
1183            insul  38.000000  38.000000  Default  26.0000  26.0000
...               ...        ...        ...      ...      ...      ...
851   friction factor   3.843584  55.430873  Topic10  -5.2466  -0.0062
1625     pin-fin heat   3.089904  29.961989  Topic10  -5.4649   0.3908
2297            veloc   2.336834  24.835874  Topic10  -5.7442   0.2991
1959            squar   2.336688  28.320860  Topic10  -5.7443   0.1677
268          coeffici   2.336660  21.872322  Topic10  -5.7443   0.4261

[718 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
7         4  0.469059  _ m mass flow
7        10  0.469059  _ m mass flow
12        6  0.490808            abl
12        8  0.490808            abl
13        7  0.483126        absolut
...     ...       ...            ...
2355      8  0.080884          yield
2355     10  0.080884          yield
2360      6  0.116776           zero
2360      8  0.817430           zero
2366      6  1.009155        zografo

[725 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# t-SNE and bokeh Visualization<a id='tsne_bokeh'></a>

We will now perform t-distributed stochasting neighbor embedding (t-SNE) on our vectorized heat exchanger articles from the previous step.  t-SNE is a non-linear means of reducing multiple dimensions to two or three dimensions for visualization.  After using t-SNE to project our vectorized SBIR documents into 2D, we will plot the documents and assign colors to them based on the previous LDA topic modeling cluster assigned.

First, we need to determine to which categories LDA assigned each document.

In [19]:
top_cluster = []
for n in range(lda_output.shape[0]):
    categ = np.argwhere(lda_output[n,:] == np.amax(lda_output[n,:]))
    top_cluster.append(str(categ[0][0]+1))

Next, we will convert our vectorized heat exchanger data from a sparse matrix to a dense one and then perform t-SNE.

In [20]:
test = data_vectorized.todense()

In [21]:
tsne = TSNE(random_state=2017, perplexity=30, n_iter=5000)#, early_exaggeration=120)
embedding = tsne.fit_transform(test)

Let's see how many unique categories were assigned to our documents during LDA topic modeling.  We originally told the algorithm to cluster the heat exchanger documents into 10 topics.  However, some of these topics were fairly small per our visualization above.  Some topics may not be assigned to any document as the primary category.

In [22]:
unique_cats = list(set(top_cluster))
unique_cats_count = len(unique_cats)
print('There are ' + str(unique_cats_count) + ' unique categories in our dataset.')

There are 5 unique categories in our dataset.


Only 10 of our categories were assigned as primary topics for our heat exchanger document collection.  A second iteration of LDA could be performed with only 6 categories in the future.  We will skip this step for now though and use our unique category count for building appropriate bokeh color palette. 

In [23]:
viridis_spec = viridis(unique_cats_count)
magma_spec = magma(unique_cats_count)
turbo_spec = linear_palette(Turbo256, unique_cats_count)

Next, let's visualize our t-SNE reduced dimensions and also use the original LDA cluster assignment for a color palette.  We will also use a hover functionality to show the index of the heat exchanger document, the document's title, and also the category # assigned to the document by the LDA clustering algorithm above.

In [24]:
source = ColumnDataSource(data=dict(
    x=embedding[:,0],
    y=embedding[:,1],
    top_cluster=top_cluster
))

TOOLTIPS = [
    ("index", '$index'),
    ("Title", '@desc'),
    ('Category', '@top_cluster')
]

In [25]:
p = figure(title="t-SNE Projection of Vectorized Heat Exchangers", tooltips=TOOLTIPS, plot_height=800, plot_width=800)
p.circle('x', 'y', size=10, source=source, fill_alpha=0.75, fill_color=factor_cmap('top_cluster', palette=magma_spec, factors=top_cluster))
show(p)

## Summary

This is a nice vizualization of the original vectorized heat exchanger documents.  It is possible to hover and see where similar documents are clustering together.  The t-SNE algorithm has projected the vectorized documents into a 2D space where similar documents should be grouped together.  For example, at roughly coordinates (7, 3), a user can hover their mouse over a darker blue grouping of heat exchanger documents.  They also all were assigned to category 4 by the original LDA clustering algorithm.  

This plot has several interactive features.  Users can scroll in and out as well as draw boxes for zooming in.  The number of colors is a bit much.  It would be desirable to rerun this notebook in the future with less color categories (and less LDA clusters).

This has been an initial exploration of leveraging scikit-learn for analysis as well as applying t-SNE to heat exchanger vectorized document and visualizing results using bokeh in Python.  For any questions on this notebook, please reach out to Charles Liles (charles.a.liles@nasa.gov) or Karsten Look (karsten.look@nasa.gov).